# «Хрупкий словарь»

В банках всего мира используются SQL-базы данных. Основным преимуществом является высокая скорость доступа к данным, а также выполнение принципов ACID (атомарность, согласованность, изолированность, долговечность). Транзакции обязаны удовлетворять этим принципам, иначе любая ошибка может привести к огромным потерям со стороны банка.

В этом задании предлагается реализовать интерфейс простейшего "безопасного" хранилища. Наше хранилище будет называться «хрупким словарем». Класс **FragileDict** должен реализовывать следующий интерфейс:


1.   Конструктор опционально может принимать на вход словарь, содержимое которого будет храниться в «хрупком словаре»; в конструкторе заполняются два атрибута: data — хранилище с данными, lock — булев флаг, показывающий, можно ли редактировать хранилище (True в положении вне контекста)

> ```python
> class FragileDict:
>     def __init__(self, d: Optional[dict]):
>         self.data = ...
>         self.lock = True
> ```


2.   Поддержка интерфейса словаря

> ```python
> d = FragileDict({"key": 5})
> assert d["key"] == 5
> assert "key" in d
> ```

3.   Класс должен поддерживать механизм менеджера контекста. Экземпляр класса может создаваться при входе в контекст

> ```python
> # example 1
> d = FragileDict({"key": 5})
> with d:
>     ...
>
> # example 2
> with FragileDict({"key": 10}) as d:
>     ...
> ```

4.   Запись данных в **FragileDict** разрешена **только** внутри контекста, иначе бросается исключение RuntimeError("Protected state"). При входе в контекст можно создавать любые атрибуты класса, но на выхоже их быть уже не должно

> ```python
> d = FragileDict({"key": 10})
>
> # должно возбуждаться исключение RuntimeError
> d["key"] = 6
>
> with d:
>     d["a"] = 20
> assert d["a"] == 20
> ```

5. Если внутри контекста возникло исключение, то данные **не записываются**. На выходе из контекста «словарь» должен иметь точно такое же состояние, как и на входе. Само исключение подавляется, и пишется сообщение об ошибке "Exception has been suppressed"


**Note:** Для обеспечения большей безопасности хранилища может пригодиться модуль *copy*. Вспомните разницу между поверхностным копированием (shallow copy) и глубоким копированием (deep copy)






Ниже представлен минимальный интерфейс класса и тесты для проверки решения

In [ ]:
!pip install ipytest

In [ ]:
from typing import Any, Optional

import pytest
import ipytest
ipytest.autoconfig()

In [ ]:
class FragileDict:
    def __init__(self, d: Optional[dict]) -> None:
        self.data = ...
        self.lock = True

    def __setitem__(self, key, value) -> None:
        ...

    def __getitem__(self, key) -> Any:
        ...

    def __contains__(self, key) -> bool:
        ...

    def __enter__(self) -> 'FragileDict':
        ...

    def __exit__(self, exc_type, exc_value, traceback) -> None:
        ...

In [1]:
def test_context_set():
    d = FragileDict({"key": 5})

    with d:
        d["key"] = 6
        d["ord"] = 7

    assert d["key"] == 6
    assert d["ord"] == 7



def test_context_set_get():
    d = FragileDict({"key": 5})

    with d:
        d["key"] = 6
        assert d["key"] == 6
        d["ord"] = 7
        assert "ord" in d
        assert d["ord"] == 7

    assert d["key"] == 6
    assert "ord" in d



def test_set_not_in_context():
    d = FragileDict({"key": 5})

    with pytest.raises(RuntimeError):
        d["key"] = 6

    with pytest.raises(RuntimeError):
        d["ord"] = 7

    # d["key"] value should be the same
    assert d["key"] == 5

    # "ord" should not be in d
    assert "ord" not in d



def test_raised_exception_in_context():
    d = FragileDict({"key": 5})

    with d:
        d["key"] = 6
        assert d["key"] == 6
        d["ord"] = 7
        assert "ord" in d
        assert d["ord"] == 7
        raise Exception()

    # data should be as before context entering
    assert d["key"] == 5
    assert "ord" not in d



def test_deepcopy_lst():
    d = FragileDict({"key": []})

    with d:
        a = d["key"]
        d["key"].append(10)
        a.append(10)

    a.append(10)

    assert a == [10, 10, 10]
    assert d["key"] == [10, 10]



def test_deepcopy_lst2():
    d = FragileDict({"key": [10, 10]})

    a = d["key"]
    a.append(10)

    assert a == [10, 10, 10]
    assert d["key"] == [10, 10]



def test_deepcopy_lst3():
    a = [10, 10]
    with FragileDict({"key": [20, 20]}) as d:
        d["a"] = a
        assert d["a"] == [10, 10]

    a.append(10)
    assert d["a"] == [10, 10]



def test_deepcopy_lst4():
    a = [10, 10]
    with FragileDict({"key": a}) as d:
        d["a"] = a
        assert d["key"] == a
        assert d["a"] == a
        a.append(10)

        assert d["key"] == [10, 10]
        assert d["a"] == a



def test_nested_good():
    with FragileDict({"a": {"b": [20, 20]}}) as a:
        # ensure correct creation
        assert a["a"] == {"b": [20, 20]}
        assert a["a"]["b"] == [20, 20]

        # change nested value
        a["a"]["b"].append(20)

        # ensure it changed
        assert a["a"]["b"] == [20, 20, 20]

    # ensure it changed after context
    assert a["a"]["b"] == [20, 20, 20]



def test_nested_bad():
    with FragileDict({"a": {"b": [10, 10]}}) as d:
        # ensure correct creation
        assert d["a"] == {"b": [10, 10]}
        assert d["a"]["b"] == [10, 10]

        # change nested value
        d["a"]["b"].append(10)

        # ensure it changed
        assert d["a"]["b"] == [10, 10, 10]

        raise Exception

    # value should be as before context
    assert d["a"]["b"] == [10, 10]



def test_nested_hard():
    a = [10, 10]
    d = FragileDict({"a": {"b": a}})
    assert "a" in d
    assert "b" not in d

    assert d["a"]["b"] == [10, 10]
    a.append(10)
    assert d["a"]["b"] == [10, 10]

In [ ]:
ipytest.run()